# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
import numpy as np

In [2]:
df = pd.read_csv('D://Игорь/projects/users_behavior.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [5]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


В датасете 5 столбцов, пропусков нет по этому предобработка не требуется

## Разбейте данные на выборки

In [6]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [7]:
#далим датасет на выборки: обучающую 60%, тестовую 20%, валидационную 20%
train_features, test_features, train_target, test_target = train_test_split(
    features, target, test_size=0.2,  stratify = target,  random_state=12345)

In [8]:
train_features, valid_features, train_target, valid_target = train_test_split(
    train_features, train_target, test_size=0.25, random_state=12345)

## Исследуйте модели

Т.к. у нас классификация, исследуем через: древо решений, случайный лес, логистическую регрессию

### Древо решений:

In [9]:

model = DecisionTreeClassifier(random_state=12345)
model.fit(train_features, train_target)
predictions = model.predict(valid_features)
accuracy_score(valid_target, predictions)

0.6889580093312597

Доля правильных ответов составила 0,71, при минимальном показателе 0,75 данного показателя недостаточно. изменим глубину дерева

In [10]:
best_tree = None
best_result = 0
for depth in range(1, 10):
    tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    tree_model.fit(train_features, train_target)
    tree_predictions = tree_model.predict(valid_features)
    result = accuracy_score(valid_target, tree_predictions)
    if result > best_result:
        best_tree = tree_model
        best_result = result
        best_depth = depth
            
print('Лучший результат:', best_result, 'с глубиной дерева', best_depth)

Лучший результат: 0.8009331259720062 с глубиной дерева 6


Получили результат 0,77 при глубине дерева 7.

### Случайный лес

In [11]:
best_forest = None
best_result = 0
for est in range(1, 50):
    forest_model = RandomForestClassifier(random_state=12345, n_estimators=est)
    forest_model.fit(train_features, train_target)
    result = forest_model.score(valid_features, valid_target)
    if result > best_result:
        best_forest = forest_model
        best_result = result
        best_est = est
print('Лучший результат:', best_result, 'с количеством деревьев', best_est)

Лучший результат: 0.7947122861586314 с количеством деревьев 16


В случайном лесе мы получили даже лучшие показатели: 0,79 с количеством деревьев 44, на текущий момент это наш фаворит

### Логистическая регрессия

In [12]:
reg_model = LogisticRegression(random_state=12345)
reg_model.fit(train_features, train_target)
result = reg_model.score(valid_features, valid_target)
print('Точность логистической регрессии:', result)

Точность логистической регрессии: 0.713841368584759


Вывод:
- Качество древа решений составляет 0,77
- Качество случайного леса составляет 0,79
- Качество логистической регрессии составляет 0,72 (не преодолела порог в 0,75)

## Проверьте модель на тестовой выборке

In [13]:
#дерево решений
print('Качество дерева решений:', best_tree.score(test_features, test_target))

#случайный лес
print('Качество случайного леса:', best_forest.score(test_features, test_target))

#логистическая регрессия
print('Качество логистической регрессии:', reg_model.score(test_features, test_target))

Качество дерева решений: 0.7729393468118196
Качество случайного леса: 0.7993779160186625
Качество логистической регрессии: 0.713841368584759


Вывод: Как и по предыдущему выводу, лучшим оказался случайный лес с результатом 0,79

## (бонус) Проверьте модели на адекватность

In [14]:
strategy = ['most_frequent', 'prior', 'stratified', 'uniform']
for index in strategy:
    dummy_clf = DummyClassifier(strategy=index, random_state=12345)
    dummy_clf.fit(train_features, train_target)
    score = dummy_clf.score(test_features, test_target)
    print('Точность для стратегии', index, ':', score)

Точность для стратегии most_frequent : 0.6936236391912908
Точность для стратегии prior : 0.6936236391912908
Точность для стратегии stratified : 0.5800933125972006
Точность для стратегии uniform : 0.4821150855365474


При проверке на простых способах показателя 0,75 не наблюдается, значит наша модель работает корректно